The ductbank model is the equilibrium thermal model (i.e., thermal mass is not considered) described by the following:

$$
    Q   \xrightarrow{} (T_C) 
        \xrightarrow{U_I} (T_S) 
        \xrightarrow{U_A} (T_I)
        \xrightarrow{U_D} (T_O)
        \xrightarrow{U_F} (T_G)
$$

where
- $Q$ is the thermal heat from the cable losses (input)
- $T_C$ is the temperature of the cable core
- $U_I$ is the thermal conductance of the cable insulation
- $T_S$ is the temperature of the cable shield
- $U_A$ is the thermal conductance of the duct air gap
- $T_I$ is the temperature of the duct inner surface
- $U_D$ is the thermal conductance of the duct core
- $T_O$ is the temperature of the duct outer surface
- $U_F$ is the thermal conductance of the fill soil
- $T_G$ is the temperature of the surface ground

The equilibrium temperatures are calculated as follows

$$
    T_O = T_G + { Q \over U_F }, \qquad
    T_I = T_O + { Q \over U_D }, \qquad
    T_S = T_I + { Q \over U_A }, \qquad
    T_C = T_S + { Q \over U_I }
$$

The conductance of the air gap is computed as follows:

1. The cross sectional area of air $A$ is the overall cross section of the duct  $A_0$ less the overall cross section of cables in the duct $A_C$, including insulation, i.e.,

$$
    A = A_0 - A_C
$$

2. The mean distance from cable to duct $D$ is one half the square root of the cross sectional area of air, i.e.,

$$
    D = { \sqrt{A} \over 2 }
$$

3. The conductance of the air gap is the effective conductivity of air at ambient conditions divided by the mean distance, i.e.,

$$
    U_A = { 0.026 \over D }
$$


The conductance of the duct is computed as follows:
1. The duct cross-sectional area is

$$
    A_D = 4 L \sqrt{A_0}
$$

where $L$ is the length of the duct.

2. The conductance of the duct is computed as follows

$$
    U_D = A_D U_{duct} 
$$

where $U_{duct}$ is composed from the table below according to the material used.

For the above calculations the following R-values and U-values may be used according the material type:

| Material | U-value (W/K.m) |
| :------- | --------------- | 
| ___Shell___
| PVC              | 0.19 |
| ___Core___       |||
| Concrete         | 0.92 |
| ___Fill soil___      |||
| Damp soil        | 1.00 |
| Average soil     | 0.80 |
| Dry soil         | 0.60 |
| ___Cable insulation___ |||
| Rubber           | 0.10 |
| Polyethylene     | 0.46 |
| __Air__|||
| Standard conditions | 0.026 |


In [1]:
from math import *
# U-value table
U = {
    "PVC" : 0.19,
    "CONCRETE" : 0.92,
    "WETSOIL" : 1.00,
    "SOIL" : 0.80,
    "DRYSOIL" : 0.60,
    "RUBBER" : 0.10,
    "PE" : 0.46,
    "AIR" : 0.20,
} # W/K.m (per-unit thickness)

# Example

Consider a duct with 6 cables of length 1000 meters, each with a loss of 1 kW in a duct bank of nominal cross sectional area 0.1 m$^2$.  The cables are 2.5 cm in diameter with 1 cm of rubber insulation.  The duct shell is 1 cm of PVC. The duct core is 40 cm thick concrete and the fill is 60 cm of average moisture soil. The ground temperature is $10^{\mathrm{o}}\mathrm{C}$.

In [97]:
# conditions as specified
N = 6 # number of cables
Q = 1 # lineal cable heat gain (W/m)
A0 = 0.01 # duct cross sectional area (m^2)
TG = 25.0 # ground temperature (degC)
DI = 0.01 # cable insulation thickness (m)
DC = 0.025 # cable diameter (M)
DD = 0.40 # duct thickness
DF = 0.60 # fill thickness

In [98]:
# insulator conductance = cable area * insulation U-value / insulation thickness
IA = N*(DC*pi) # insulation area (m^2)
UI = U["RUBBER"] / DI * IA # W/K

In [99]:
# air conductance
AC = N*(DC/4)**2*pi # cable cross sectional area (m^2)
A = A0-AC # air cross sectional area (m^2)
if A <= 0: raise Exception("duct is too small")
D = sqrt(A)/2 # mean air gap from cable to duct (m)
UA = U["AIR"]/D # W/K
UA

4.1559278843836225

In [100]:
# duct conductance
UD = U["CONCRETE"]/DD # Concrete duct thickness DD
UD

2.3

In [101]:
# fill conductance
UF = U["DRYSOIL"]/DF # soil thickness DF
UF 

1.0

In [102]:
# temperature gradient
US = 1/(1/UD+1/UF)
TD = TG + N*Q/US
TS = TD + N*Q/UA
TC = TS + N*Q/UI
print('                       ','           '.
      join(['%2s=%-5.2f'%(x,globals()[x]) for x in ['UI','UA','US']]))
print('Q=%-5.0f --->'%(N*Q),' ------> '.
      join(["(%2s=%-5.1f)"%(x,globals()[x]) for x in ["TC","TS","TD","TG"]]))


                        UI=4.71            UA=4.16            US=0.70 
Q=6     ---> (TC=36.3 ) ------> (TS=35.1 ) ------> (TD=33.6 ) ------> (TG=25.0 )


# Validation

In [66]:
# Typical SCE 4x2 Duct Bank
N = 3 # cables
M = 6 # channels
A0 = M*(5*0.0254/2)**2*pi # 5 inch channel
TG = 25 # degC
DI = 1.65*0.0254 # 1.65 inch insulation
DC = 2.0236*0.0254 # overall cable diameter
DD = 3.0*0.0254 # concrete thickness
DF = 3.0*0.0254 # fill thickness
Q = (0.97/1600)**2 * 650 # R=0.97 Ohm/mile at 650 A

In [54]:
# insulator conductance = cable area * insulation U-value / insulation thickness
IA = N*(DC*pi) # insulation area (m^2)
UI = U["PE"] / DI * IA # W/K

In [65]:
# air conductance
AC = N*(DC/4)**2*pi # cable cross sectional area (m^2)
A = (A0-AC)/M # air cross sectional area per duct (m^2)
if A <= 0: raise Exception("duct is too small")
D = sqrt(A)/2 # mean air gap from cable to duct (m)
AD = 4*sqrt(A) # effective air surface area
UA = U["AIR"]/D*AD # W/K

In [56]:
# duct conductance
Ucore = U["CONCRETE"]/DD # Concrete duct thickness
Ufill = U["DRYSOIL"]/DF
US = AD*(1/Ufill+1/Ucore) # W/K

In [57]:
# temperature gradient
TD = TG + N*Q/US
TS = TD + N*Q/UA
TC = TS + N*Q/UI
print('                       ','           '.
      join(['%2s=%-5.2f'%(x,globals()[x]) for x in ['UI','UA','US']]))
print('Q=%-5.0f --->'%(N*Q),' ------> '.
      join(["(%2s=%-5.1f)"%(x,globals()[x]) for x in ["TC","TS","TD","TG"]]))


                        UI=5.32            UA=1.60            US=0.09 
Q=0     ---> (TC=25.0 ) ------> (TS=25.0 ) ------> (TD=25.0 ) ------> (TG=25.0 )
